In [ ]:
import scvelo as scv
import scanpy as sc
import cellrank as cr
import numpy as np
import pandas as pd
import anndata as ad
from scipy import io
from scipy.sparse import coo_matrix, csr_matrix
import os
import anndata

In [ ]:
sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=92,
    dpi_save=300,
    facecolor="white",
    frameon=False,
)

# Construct the scRNA-seq scanpy anndata object

In [ ]:
# load sparse matrix:
X = io.mmread("counts.mtx")

# create anndata object
adata = anndata.AnnData(
    X=X.transpose().tocsr()
)

# load cell metadata:
cell_meta = pd.read_csv("metadata.csv")

# load gene names:
with open("gene_names.csv", 'r') as f:
    gene_names = f.read().splitlines()

# set anndata observations and index obs by barcodes, var by gene names
adata.obs = cell_meta
adata.obs.index = adata.obs['barcodes']
adata.var.index = gene_names

# load dimensional reduction:
pca = pd.read_csv("pca.csv")
pca.index = adata.obs.index

harmony = pd.read_csv("harmony.csv")
harmony.index = adata.obs.index

# set pca and umap
#adata.obsm['X_pca'] = pca.to_numpy()
adata.obsm['X_pca'] = harmony.to_numpy()
#adata.obsm['X_harmony'] = harmony.to_numpy()
adata.obsm['X_umap'] = np.vstack((adata.obs['UMAP_1'].to_numpy(), adata.obs['UMAP_2'].to_numpy())).T


In [ ]:
adata

In [ ]:
cols_to_drop = [col for col in adata.obs.columns 
                if col.startswith(('pANN', 'DF','RNA_snn_res'))]
adata.obs.drop(columns=cols_to_drop, inplace=True)

In [ ]:
adata

In [ ]:
adata.obs['seurat_clusters'] = 'C' + adata.obs['seurat_clusters'].astype('str')

In [ ]:
# plot a UMAP colored by sampleID to test:
sc.pl.umap(adata, color=['seurat_clusters'], frameon=True, save=True, title = "Subtype")

In [ ]:
# save dataset as anndata format
adata.write('scRNAseq_anno.h5ad')

In [ ]:
adata = sc.read_h5ad("scRNAseq_anno.h5ad")

# Loading and Constructing spliced and unspliced counts matrices from mutiple sample loom file

In [ ]:
scv.settings.verbosity = 3
scv.settings.set_figure_params('scvelo', facecolor='white', dpi=100,dpi_save = 300, frameon=False, format="pdf")
cr.settings.verbosity = 2

In [ ]:
sample_info = pd.read_csv("all_sample_info.txt", sep = "\t")
type(sample_info)
sample_info

In [ ]:
ladatas = {}
from datetime import datetime

for i in sample_info['samplename']:
    sample_name = i

    # -----------------------------start
    now_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print("[" + now_time + "] >>> " + "Start Processing of " + sample_name + " ......")
    
    # get sampleinfo for each sample
    sample_type = sample_info.loc[sample_info['samplename'] == sample_name]['sample_type'].values[0]
    group = sample_info.loc[sample_info['samplename'] == sample_name]['group'].values[0]
    sampling_time = sample_info.loc[sample_info['samplename'] == sample_name]['sampling_time'].values[0]
    loompath = sample_info.loc[sample_info['samplename'] == sample_name]['velocyto_loompath'].values[0]


    # read matrixfile
    sample_adata = sc.read(loompath, cache=True)
    sample_adata.var_names_make_unique()
    
    sample_adata.obs['barcodes'] = [sample_name + "_" + bc[0:len(bc)-1] + '-1' for bc in [bc.split(':')[1] for bc in sample_adata.obs.index.to_list()]]
    
    sample_adata.obs['sample_name'] = sample_name
    sample_adata.obs['sampling_time'] = sampling_time
    sample_adata.obs['sample_type'] = sample_type
    sample_adata.obs['group'] = group

    #---------------------------------done
    now_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print("[" + now_time + "] >>> " + "Sample `" + sample_name + "` Analysis is Done!\n\n")
     
    ladatas[sample_name] = sample_adata
    

In [ ]:
#ladata = ad.concat(ladatas, label="sample_batch")
ladata_list = list(ladatas.values())
ladata = ladata_list[0].concatenate(ladata_list[1:len(ladata_list)])


In [ ]:
ladata.obs.index = ladata.obs['barcodes'].to_list()

In [ ]:
ladata.obs.head(3)

In [ ]:
adata.obs.head(3)

In [ ]:
adata.obs.index = adata.obs['barcodes'].to_list()

In [ ]:
# merge matrices into the original adata object
adata_merge = scv.utils.merge(adata, ladata)

In [ ]:
adata_merge

In [ ]:
# plot umap to check
from matplotlib.pyplot import rc_context
import matplotlib.pyplot as plt

with rc_context({"figure.figsize": (5, 5)}):
    sc.pl.umap(adata_merge, color='celltype', palette='tab20b', title = 'Subtype', size = 10, legend_fontsize=12, frameon=True, show=False)
    plt.savefig('./figures/Velocyto_subcelltypes_Umap.pdf', format='pdf', bbox_inches='tight')
    plt.show()
    

## scVelo

In [ ]:
adata_td = adata_merge[adata_merge.obs['td'] == 'td']

In [ ]:
# plot umap to check
from matplotlib.pyplot import rc_context
import matplotlib.pyplot as plt

with rc_context({"figure.figsize": (5, 5)}):
    sc.pl.umap(adata_td, color='celltype', palette='tab20b', title = 'Subtype', size = 10, legend_fontsize=12, frameon=True, show=False)
    plt.savefig('./figures/COPD_tdTomato_Velocyto_Basal_Subtype_Umap.pdf', format='pdf', bbox_inches='tight')
    plt.show()
    

In [ ]:
adata = adata_td

In [ ]:
adata

In [ ]:
from matplotlib.pyplot import rc_context
import matplotlib.pyplot as plt

with rc_context({"figure.figsize": (15, 6)}):
    scv.pl.proportions(adata, groupby = 'celltype', show=False)
    plt.savefig('./figures/velocyto_summary.pdf', format='pdf', bbox_inches='tight')
    plt.show()
    


## Preprocess the Data

In [ ]:
# Prepocess data
scv.pp.filter_genes(adata, min_shared_counts=20)
scv.pp.normalize_per_cell(adata)
scv.pp.filter_genes_dispersion(adata, n_top_genes=2000)
scv.pp.log1p(adata)

In [ ]:
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)


## Estimate RNA velocity

In [ ]:
scv.tl.recover_dynamics(adata)

In [ ]:
scv.tl.velocity(adata, mode='dynamical')
scv.tl.velocity_graph(adata)

In [ ]:
# plot umap to check
from matplotlib.pyplot import rc_context
import matplotlib.pyplot as plt

with rc_context({"figure.figsize": (5, 5)}):
    scv.pl.velocity_embedding_stream(adata, basis='umap', color="celltype", legend_loc="right", show=False)
    plt.savefig("./figures/cVelo_streamplot.svg",bbox_inches='tight', dpi=300)

## Kinetic rate paramters

In [ ]:
df = adata.var
df = df[(df['fit_likelihood'] > .1) & df['velocity_genes'] == True]

kwargs = dict(xscale='log', fontsize=16)
with scv.GridSpec(ncols=3) as pl:
    pl.hist(df['fit_alpha'], xlabel='transcription rate', **kwargs)
    pl.hist(df['fit_beta'] * df['fit_scaling'], xlabel='splicing rate', xticks=[.1, .4, 1], **kwargs)
    pl.hist(df['fit_gamma'], xlabel='degradation rate', xticks=[.1, .4, 1], **kwargs)

scv.get_df(adata, 'fit*', dropna=True).head()


## Latent time

In [ ]:
scv.tl.latent_time(adata)

In [ ]:
with rc_context({"figure.figsize": (6, 6)}):
    scv.pl.scatter(adata, color='latent_time', color_map='coolwarm', size = 5, show=False)
    plt.subplots_adjust(left=0.15, right=0.85) 
    plt.savefig("./figures/COPD_scVelo_LatentTime.pdf", dpi=300)
    plt.show()
    plt.close()

In [ ]:
sc.tl.paga(adata, groups="celltype")

In [ ]:
adata.write('scVelo_res.h5ad', compression='gzip')